In [14]:
from google.cloud import bigquery
from google.cloud import bigquery_storage
creds = "/Users/srdeo/OneDrive - Copart, Inc/secrets/cprtpr-datastewards-sp1-614d7e297848 (1).json"
client = bigquery.Client.from_service_account_json(json_credentials_path=creds)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=creds)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [15]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

In [3]:
from datetime import datetime, timedelta
import pandas as pd

# Generate list of month starts from Nov 2024 to April 2025
month_starts = pd.date_range(start='2024-11-01', end='2025-04-01', freq='MS')

query_parts = []

for start_date in month_starts:
    end_date = (start_date + pd.offsets.MonthEnd(1)).date()
    month_label = start_date.strftime('%Y-%m')
    start_str = start_date.strftime('%Y-%m-%d')
    end_str = end_date.strftime('%Y-%m-%d')

    # Combined block with CASE WHEN for churn
    query = f"""
    SELECT
        m.owner_mbr_nbr,
        m.mbr_nbr,
        m.mbr_type,
        m.mbr_mbrshp_type_cd,
        e.event_type,
        e.event_value,
        DATE_TRUNC(e.event_value, MONTH) AS event_month,
        '{month_label}' AS month_label,
        CASE
            WHEN m.mbr_type = 'LPSDPREMIER' AND e.event_type = 'MEMBER TYPE UPDATED FROM PREMIER TO GUEST' THEN 1
            WHEN m.mbr_type = 'PREMIER' AND e.event_type = 'Member Premier Renewal Paid Date' THEN 0
            ELSE NULL
        END AS churn
    FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact m
    JOIN `cprtpr-dataplatform-sp1`.usmart.v_us_member_events_fact e
        ON m.mbr_nbr = e.mbr_nbr
    WHERE (
        (m.mbr_type = 'PREMIER' AND e.event_type = 'Member Premier Renewal Paid Date') OR
        (m.mbr_type = 'LPSDPREMIER' AND e.event_type = 'MEMBER TYPE UPDATED FROM PREMIER TO GUEST')
    )
      AND e.event_value BETWEEN '{start_str}' AND '{end_str}'
    """

    query_parts.append(query)

# Combine all parts with UNION ALL
final_query = "\nUNION ALL\n".join(query_parts)

# Run the query
data1 = client.query(final_query).to_dataframe()

In [4]:
data1
print(data1.shape)
print(data1.isnull().sum())
print(data1['churn'].value_counts())
print(data1['month_label'].value_counts().sort_index())

,owner_mbr_nbr,mbr_nbr,mbr_type,mbr_mbrshp_type_cd,event_type,event_value,event_month,month_label,churn
0,878062,878062,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2025-04-20 10:00:20+00:00,2025-04-01 00:00:00+00:00,2025-04,1
1,137927,137927,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2025-04-14 10:00:12+00:00,2025-04-01 00:00:00+00:00,2025-04,1
2,69027,69027,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2025-04-01 10:00:15+00:00,2025-04-01 00:00:00+00:00,2025-04,1
3,851283,851283,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2025-04-14 10:00:12+00:00,2025-04-01 00:00:00+00:00,2025-04,1
4,109526,109526,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2025-04-13 10:00:17+00:00,2025-04-01 00:00:00+00:00,2025-04,1
...,...,...,...,...,...,...,...,...,...
42673,221074,221074,PREMIER,PREMIER,Member Premier Renewal Paid Date,2025-03-14 16:31:09+00:00,2025-03-01 00:00:00+00:00,2025-03,0
42674,218513,218513,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2025-03-04 11:00:44+00:00,2025-03-01 00:00:00+00:00,2025-03,1
42675,278310,278310,PREMIER,PREMIER,Member Premier Renewal Paid Date,2025-03-02 09:00:13+00:00,2025-03-01 00:00:00+00:00,2025-03,0
42676,210451,210451,PREMIER,PREMIER,Member Premier Renewal Paid Date,2025-03-19 19:03:57+00:00,2025-03-01 00:00:00+00:00,2025-03,0


(42678, 9)
owner_mbr_nbr         0
mbr_nbr               0
mbr_type              0
mbr_mbrshp_type_cd    0
event_type            0
event_value           0
event_month           0
month_label           0
churn                 0
dtype: int64
churn
0    22883
1    19795
Name: count, dtype: Int64
month_label
2024-11    5130
2024-12    8397
2025-01    8422
2025-02    6623
2025-03    7692
2025-04    6414
Name: count, dtype: int64


In [5]:
# Ensure 'month_label' is in string format like '2024-05'
unique_months = data1['month_label'].unique()

for month in unique_months:
    # Filter for that month
    df_month = data1[data1['month_label'] == month]

    # Extract lists
    churn_list = df_month[df_month['churn'] == 1]['owner_mbr_nbr'].unique().tolist()
    not_churn_list = df_month[df_month['churn'] == 0]['owner_mbr_nbr'].unique().tolist()

    # Create valid variable names: 'churn_members_may_2024', etc.
    var_suffix = pd.to_datetime(month).strftime('%B_%Y').lower()

    # Store in dynamically named lists
    globals()[f"churn_members_{var_suffix}"] = churn_list
    globals()[f"not_churn_members_{var_suffix}"] = not_churn_list


In [6]:
# Reuse the same month list from data1
unique_months = data1['month_label'].unique()

for month in unique_months:
    var_suffix = pd.to_datetime(month).strftime('%B_%Y').lower()

    # Get churn and not churn lists by name
    churn_members = globals().get(f"churn_members_{var_suffix}", [])
    not_churn_members = globals().get(f"not_churn_members_{var_suffix}", [])

    # Combine and deduplicate
    all_members = pd.unique(churn_members + not_churn_members).tolist()

    # Save as new variable
    globals()[f"all_members_{var_suffix}"] = all_members

In [7]:
# Loop through each month's member list
unique_months = data1['month_label'].unique()

for month in unique_months:
    var_suffix = pd.to_datetime(month).strftime('%B_%Y').lower()

    # Get all_members list for this month
    all_members = globals().get(f"all_members_{var_suffix}", [])

    # Convert to list of strings (no quotes)
    all_members_list = [str(x) for x in all_members]

    # Join into comma-separated string
    all_members_str = ", ".join(all_members_list)

    # Store both
    globals()[f"all_members_list_{var_suffix}"] = all_members_list
    globals()[f"all_members_str_{var_suffix}"] = all_members_str

* Below query extracts data for churned members in the last 6 months

In [10]:
all_churn_members = (
    churn_members_november_2024 +
    churn_members_december_2024 +
    churn_members_january_2025 +
    churn_members_february_2025 +
    churn_members_march_2025 +
    churn_members_april_2025
)

# Same for all members (churned + non-churned) for those months
all_members_combined_str = ", ".join([
    all_members_str_november_2024,
    all_members_str_december_2024,
    all_members_str_january_2025,
    all_members_str_february_2025,
    all_members_str_march_2025,
    all_members_str_april_2025
])

query = f"""
WITH lot_fact_filtered AS (
  SELECT *
  FROM `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact
  WHERE inv_dt BETWEEN '2024-05-01' AND '2025-03-31'
    AND seller_parent_company != 'CDS'
),

member_churn_info AS (
  SELECT owner_mbr_nbr, mbr_nbr, mbr_state, mbr_country, member_sgmnt_type
  FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact
  WHERE owner_mbr_nbr IN ({all_members_combined_str})
),

bids_placed AS (
  SELECT
    m.owner_mbr_nbr,
    m.mbr_state,
    m.mbr_country,
    m.member_sgmnt_type,
    COUNTIF(l.inv_dt IS NOT NULL) AS total_bids_on_lots_sold
  FROM member_churn_info m
  LEFT JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
    ON m.mbr_nbr = b.buyer_nbr
  LEFT JOIN lot_fact_filtered l
    ON b.lot_nbr = l.lot_nbr
  GROUP BY m.owner_mbr_nbr, m.mbr_state, m.mbr_country, m.member_sgmnt_type
),

lots_bid_on AS (
  SELECT
    b.buyer_nbr AS owner_mbr_nbr,
    COUNT(DISTINCT b.lot_nbr) AS total_lots_placed_bids_on
  FROM `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
  JOIN lot_fact_filtered l
    ON b.lot_nbr = l.lot_nbr
  GROUP BY b.buyer_nbr
),

lots_won AS (
  SELECT
    l.buyer_nbr AS mbr_nbr,
    COUNT(DISTINCT l.lot_nbr) AS total_lots_won
  FROM lot_fact_filtered l
  GROUP BY l.buyer_nbr
),

median_sp_cte AS (
  SELECT
    l.buyer_nbr AS mbr_nbr,
    APPROX_QUANTILES(l.high_bid_amt, 2)[OFFSET(1)] AS median_sp_won
  FROM lot_fact_filtered l
  GROUP BY l.buyer_nbr
),

lost_bid_stats AS (
  SELECT
    b.buyer_nbr AS mbr_nbr,
    APPROX_QUANTILES(l.high_bid_amt, 2)[OFFSET(1)] AS median_sp_lost,
    MAX(b.bid_amt) AS max_bid_amt_lost
  FROM `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
  JOIN lot_fact_filtered l
    ON b.lot_nbr = l.lot_nbr
  WHERE l.buyer_nbr != b.buyer_nbr
  GROUP BY b.buyer_nbr
)

SELECT
  bp.owner_mbr_nbr,
  bp.total_bids_on_lots_sold,
  lb.total_lots_placed_bids_on,
  w.total_lots_won,
  m.median_sp_won,
  l.median_sp_lost,
  l.max_bid_amt_lost,
  bp.mbr_state,
  bp.mbr_country,
  bp.member_sgmnt_type
FROM bids_placed bp
LEFT JOIN lots_bid_on lb ON bp.owner_mbr_nbr = lb.owner_mbr_nbr
LEFT JOIN lots_won w ON bp.owner_mbr_nbr = w.mbr_nbr
LEFT JOIN median_sp_cte m ON bp.owner_mbr_nbr = m.mbr_nbr
LEFT JOIN lost_bid_stats l ON bp.owner_mbr_nbr = l.mbr_nbr
"""

# Run the optimized query
final_df = client.query(query).to_dataframe()

# Add churn tag
final_df['churn'] = final_df['owner_mbr_nbr'].isin(all_churn_members).astype(int)

In [11]:
final_df

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,median_sp_won,median_sp_lost,max_bid_amt_lost,mbr_state,mbr_country,member_sgmnt_type,churn
0,345204,138,38,7,3300.0,3300.0,7200.0,CA,USA,General Business,0
1,498176,6966,1487,189,2750.0,3850.0,10400.0,HN,HND,Exporter,0
2,109663,6755,1546,255,550.0,1250.0,33000.0,PA,USA,Dealer,0
3,602676,1618,456,56,225.0,600.0,2850.0,LA,USA,Dealer,0
4,672794,7157,1971,716,500.0,750.0,12700.0,IN,USA,Dismantler,0
...,...,...,...,...,...,...,...,...,...,...,...
42580,337398,118,23,1,875.0,2200.0,7400.0,PA,USA,Consumer,1
42581,543973,251,23,<NA>,NaN,13300.0,48250.0,FL,USA,Consumer,1
42582,403914,119,23,2,1600.0,2700.0,4750.0,TX,USA,Consumer,1
42583,694706,153,23,14,1400.0,3850.0,10100.0,FL,USA,Dealer,0


In [13]:
from datetime import datetime
import pandas as pd
import numpy as np

# Define the churn months
month_range = [ "2025-04"
]

all_dfs = []

for churn_month in month_range:
    churn_dt = datetime.strptime(churn_month + "-01", "%Y-%m-%d")
    month_key = churn_dt.strftime('%B').lower() + '_' + str(churn_dt.year)

    # Fetch member data from global scope
    churn_members = globals()[f'churn_members_{month_key}']
    all_members_str = globals()[f'all_members_str_{month_key}']

    query = f"""
    WITH lot_fact_filtered AS (
      SELECT *
      FROM `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact
      WHERE inv_dt BETWEEN '2024-10-01' AND '2025-03-31'
        AND seller_parent_company != 'CDS'
    ),

    member_churn_info AS (
      SELECT owner_mbr_nbr, mbr_nbr, mbr_state, mbr_country, member_sgmnt_type
      FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact
      WHERE owner_mbr_nbr IN ({all_members_str})
    ),

    bids_placed AS (
      SELECT
        m.owner_mbr_nbr,
        m.mbr_state,
        m.mbr_country,
        m.member_sgmnt_type,
        COUNTIF(l.inv_dt IS NOT NULL) AS total_bids_on_lots_sold
      FROM member_churn_info m
      LEFT JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
        ON m.mbr_nbr = b.buyer_nbr
      LEFT JOIN lot_fact_filtered l
        ON b.lot_nbr = l.lot_nbr
      GROUP BY m.owner_mbr_nbr, m.mbr_state, m.mbr_country, m.member_sgmnt_type
    ),

    lots_bid_on AS (
      SELECT
        b.buyer_nbr AS owner_mbr_nbr,
        COUNT(DISTINCT b.lot_nbr) AS total_lots_placed_bids_on
      FROM `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
      JOIN lot_fact_filtered l
        ON b.lot_nbr = l.lot_nbr
      GROUP BY b.buyer_nbr
    ),

    lots_won AS (
      SELECT
        l.buyer_nbr AS mbr_nbr,
        COUNT(DISTINCT l.lot_nbr) AS total_lots_won
      FROM lot_fact_filtered l
      GROUP BY l.buyer_nbr
    ),

    median_sp_cte AS (
      SELECT
        l.buyer_nbr AS mbr_nbr,
        APPROX_QUANTILES(l.high_bid_amt, 2)[OFFSET(1)] AS median_sp_won
      FROM lot_fact_filtered l
      GROUP BY l.buyer_nbr
    ),

    lost_bid_stats AS (
      SELECT
        b.buyer_nbr AS mbr_nbr,
        APPROX_QUANTILES(l.high_bid_amt, 2)[OFFSET(1)] AS median_sp_lost,
        MAX(b.bid_amt) AS max_bid_amt_lost
      FROM `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
      JOIN lot_fact_filtered l
        ON b.lot_nbr = l.lot_nbr
      WHERE l.buyer_nbr != b.buyer_nbr
      GROUP BY b.buyer_nbr
    )

    SELECT
      bp.owner_mbr_nbr,
      bp.total_bids_on_lots_sold,
      lb.total_lots_placed_bids_on,
      w.total_lots_won,
      m.median_sp_won,
      l.median_sp_lost,
      l.max_bid_amt_lost,
      bp.mbr_state,
      bp.mbr_country,
      bp.member_sgmnt_type
    FROM bids_placed bp
    LEFT JOIN lots_bid_on lb ON bp.owner_mbr_nbr = lb.owner_mbr_nbr
    LEFT JOIN lots_won w ON bp.owner_mbr_nbr = w.mbr_nbr
    LEFT JOIN median_sp_cte m ON bp.owner_mbr_nbr = m.mbr_nbr
    LEFT JOIN lost_bid_stats l ON bp.owner_mbr_nbr = l.mbr_nbr
    """

    # Run the query
    df = client.query(query).to_dataframe()

    # Post-processing
    df['churn'] = df['owner_mbr_nbr'].isin(churn_members).astype(int)
    df['bid_atleast_once'] = np.where(df['total_lots_placed_bids_on'].fillna(0) > 0, 1, 0)
    df['won_atleast_once'] = np.where(pd.isna(df['total_lots_won']) | (df['total_lots_won'] == 0), 0, 1)
    df.insert(0, 'month', churn_dt.strftime('%B_%Y'))  # e.g., 'November_2024'

    all_dfs.append(df)

# Combine all months into final DataFrame
final_combined_df = pd.concat(all_dfs, ignore_index=True)

KeyboardInterrupt: 

In [18]:
query = f"""
    WITH lot_fact_filtered AS (
      SELECT *
      FROM `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact
      WHERE inv_dt BETWEEN '2025-01-01' AND '2025-03-31'
        AND seller_parent_company != 'CDS'
    ),

    member_info AS (
      SELECT owner_mbr_nbr, mbr_nbr, mbr_state, mbr_country, member_sgmnt_type
      FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact
      WHERE mbr_type = 'PREMIER'
    ),

    bids_placed AS (
      SELECT
        m.owner_mbr_nbr,
        m.mbr_state,
        m.mbr_country,
        m.member_sgmnt_type,
        COUNTIF(l.inv_dt IS NOT NULL) AS total_bids_on_lots_sold
      FROM member_info m
      LEFT JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
        ON m.mbr_nbr = b.buyer_nbr
      LEFT JOIN lot_fact_filtered l
        ON b.lot_nbr = l.lot_nbr
      GROUP BY m.owner_mbr_nbr, m.mbr_state, m.mbr_country, m.member_sgmnt_type
    ),

    lots_bid_on AS (
      SELECT
        b.buyer_nbr AS owner_mbr_nbr,
        COUNT(DISTINCT b.lot_nbr) AS total_lots_placed_bids_on
      FROM `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
      JOIN lot_fact_filtered l
        ON b.lot_nbr = l.lot_nbr
      GROUP BY b.buyer_nbr
    ),

    lots_won AS (
      SELECT
        l.buyer_nbr AS mbr_nbr,
        COUNT(DISTINCT l.lot_nbr) AS total_lots_won
      FROM lot_fact_filtered l
      GROUP BY l.buyer_nbr
    )

    SELECT
      bp.owner_mbr_nbr,
      bp.total_bids_on_lots_sold,
      lb.total_lots_placed_bids_on,
      w.total_lots_won,
      bp.mbr_state,
      bp.mbr_country,
      bp.member_sgmnt_type
    FROM bids_placed bp
    LEFT JOIN lots_bid_on lb ON bp.owner_mbr_nbr = lb.owner_mbr_nbr
    LEFT JOIN lots_won w ON bp.owner_mbr_nbr = w.mbr_nbr
    """

    # Run the query
df = client.query(query).to_dataframe()

In [19]:
df

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,mbr_state,mbr_country,member_sgmnt_type
0,327115,49,11,3,PN,PAN,Exporter
1,935736,0,<NA>,<NA>,PL,POL,Exporter
2,376187,0,<NA>,<NA>,NL,MEX,Exporter
3,288680,1006,88,27,PL,POL,Exporter
4,398204,0,<NA>,<NA>,HI,USA,Consumer
...,...,...,...,...,...,...,...
96253,615384,17,2,1,WI,USA,Consumer
96254,239571,62,10,<NA>,WI,USA,Dealer
96255,969925,0,<NA>,<NA>,WI,USA,Consumer
96256,946628,2403,266,10,WI,USA,Dealer


In [23]:
df['winning_perc'] = np.round((df['total_lots_won'] / df['total_lots_placed_bids_on'])*100,2)

In [24]:
df

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,mbr_state,mbr_country,member_sgmnt_type,winning_perc
0,327115,49,11,3,PN,PAN,Exporter,27.27
1,935736,0,<NA>,<NA>,PL,POL,Exporter,<NA>
2,376187,0,<NA>,<NA>,NL,MEX,Exporter,<NA>
3,288680,1006,88,27,PL,POL,Exporter,30.68
4,398204,0,<NA>,<NA>,HI,USA,Consumer,<NA>
...,...,...,...,...,...,...,...,...
96253,615384,17,2,1,WI,USA,Consumer,50.0
96254,239571,62,10,<NA>,WI,USA,Dealer,<NA>
96255,969925,0,<NA>,<NA>,WI,USA,Consumer,<NA>
96256,946628,2403,266,10,WI,USA,Dealer,3.76


In [26]:
df[df['winning_perc']<50]

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,mbr_state,mbr_country,member_sgmnt_type,winning_perc
0,327115,49,11,3,PN,PAN,Exporter,27.27
3,288680,1006,88,27,PL,POL,Exporter,30.68
5,921077,9,3,1,NS,CAN,Exporter,33.33
8,153865,4800,1039,171,JL,MEX,Exporter,16.46
9,666750,237,71,7,LY,LBY,Exporter,9.86
...,...,...,...,...,...,...,...,...
96248,484157,70,6,1,WI,USA,Dealer,16.67
96249,556222,2663,316,39,WI,USA,Dealer,12.34
96250,365083,686,140,26,WI,USA,Dealer,18.57
96252,807407,137,33,6,WI,USA,Dealer,18.18


In [53]:
df_mbr_lst = df['owner_mbr_nbr'].unique().tolist()

In [58]:
query1 = """
select b.lot_nbr, m.owner_mbr_nbr, b.buyer_nbr, m.mbr_country, b.bid_type_desc, b.auc_dt, l.proquote_amt_new, l.proquote_amt
from `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact m
join `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
    on m.mbr_nbr = b.buyer_nbr
left join `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
on b.lot_nbr = l.lot_nbr
where b.auc_dt BETWEEN '2025-07-16' and '2025-07-22' or b.bid_type_desc is NULL
and l.seller_parent_company!='CDS'
and m.mbr_type = 'PREMIER'
"""

data1 = client.query(query1).to_dataframe()


In [59]:
data1

,lot_nbr,owner_mbr_nbr,buyer_nbr,mbr_country,bid_type_desc,auc_dt,proquote_amt_new,proquote_amt
0,57037105,248267,248267,USA,Preliminary Bid,2025-07-16,8349.49,8349.49
1,57037105,248267,248267,USA,Preliminary Bid,2025-07-16,8349.49,8349.49
2,57037105,248267,248267,USA,Preliminary Bid,2025-07-16,8349.49,8349.49
3,47654005,587877,587877,USA,Preliminary Bid,2025-07-17,1619.38,1619.38
4,53405915,680880,680880,USA,Preliminary Bid,2025-07-16,6461.41,6461.41
...,...,...,...,...,...,...,...,...
114117,58211535,883611,883611,USA,Preliminary Bid,2025-07-16,2901.90,2901.90
114118,58211535,883611,883611,USA,Preliminary Bid,2025-07-16,2901.90,2901.90
114119,58211535,883611,883611,USA,Preliminary Bid,2025-07-16,2901.90,2901.90
114120,62651295,680880,680880,USA,Preliminary Bid,2025-07-16,5541.77,5541.77


In [60]:
data1['lot_nbr'].nunique()

25685

In [61]:
data1.groupby('bid_type_desc')['lot_nbr'].nunique().reset_index(name = 'unique_lot_count')

,bid_type_desc,unique_lot_count
0,Preliminary Bid,25685


In [65]:
data1[data1['owner_mbr_nbr'].isin(df_mbr_lst)]

,lot_nbr,owner_mbr_nbr,buyer_nbr,mbr_country,bid_type_desc,auc_dt,proquote_amt_new,proquote_amt
0,57037105,248267,248267,USA,Preliminary Bid,2025-07-16,8349.49,8349.49
1,57037105,248267,248267,USA,Preliminary Bid,2025-07-16,8349.49,8349.49
2,57037105,248267,248267,USA,Preliminary Bid,2025-07-16,8349.49,8349.49
3,47654005,587877,587877,USA,Preliminary Bid,2025-07-17,1619.38,1619.38
5,64468965,165810,165810,USA,Preliminary Bid,2025-07-17,9539.72,9539.72
...,...,...,...,...,...,...,...,...
114109,58364205,739482,739482,GTM,Preliminary Bid,2025-07-17,2277.99,2277.99
114116,58211535,883611,883611,USA,Preliminary Bid,2025-07-16,2901.90,2901.90
114117,58211535,883611,883611,USA,Preliminary Bid,2025-07-16,2901.90,2901.90
114118,58211535,883611,883611,USA,Preliminary Bid,2025-07-16,2901.90,2901.90
